In [ ]:
import os
import time
import shutil
import pathlib
import itertools
from PIL import Image
import random

# import data handling tools
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
# import Deep learning Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras import regularizers

# Ignore Warnings

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="urllib3")

print ('modules loaded')

In [ ]:
import cv2
import os

# Path to the main folder containing images
base_folder = "/Users/kisanetgabreselassie/Downloads/archive 2"

# Define the sets of folders to iterate over
sets = ['test', 'train', 'valid']
categories = ['nowildfire', 'wildfire']

# Loop over each set and category to read images
for dataset in sets:
    for category in categories:
        category_path = os.path.join(base_folder, dataset, category)

        # Load all images from the category folder
        if os.path.exists(category_path):
            image_files = os.listdir(category_path)
            for image_file in image_files:
                image_path = os.path.join(category_path, image_file)

                # Read and display image
                image = cv2.imread(image_path)

                # Check if the image was loaded correctly
                if image is not None:
                    print(f"Image: {image_file} from {dataset}/{category}")
                    print(f"Dimensions: {image.shape[:2]} (Height x Width)")
                    print(f"Channels: {image.shape[2] if len(image.shape) == 3 else 1}")
                    # Uncomment the next lines to display each image
                    # cv2.imshow("Image", image)
                    # cv2.waitKey(0)
                else:
                    print(f"Could not load image {image_file} from {dataset}/{category}")

# Close any open image windows
cv2.destroyAllWindows()


In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the base folder path
base_folder = "/Users/kisanetgabreselassie/Downloads/archive 2"

# Path to datasets
train_dir = os.path.join(base_folder, 'train')
valid_dir = os.path.join(base_folder, 'valid')
test_dir = os.path.join(base_folder, 'test')

# Set up ImageDataGenerator for data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1.0/255)

# Load images from train and validation directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),  # Adjust based on your model input size
    batch_size=32,
    class_mode='binary'  # Assuming binary classification (wildfire vs nowildfire)
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='binary'
)

print("Data Generators ready for training.")


In [ ]:
from tensorflow.keras import layers, models

model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Output layer for binary classification
])


In [ ]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=15,  # Adjust the number of epochs as needed
    validation_data=valid_generator,
    validation_steps=len(valid_generator)
)


In [ ]:
model.save('wildfire_prediction_model.h5')

loss, accuracy = model.evaluate(valid_generator)
print(f"Validation Loss: {loss}")
print(f"Validation Accuracy: {accuracy}")